In [1]:
import os
import numpy as np
import pywt
import random
import seaborn as sns
import tensorflow as tf
from keras.layers import Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
  Input, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, 
  Dense, Dropout, GRU, Concatenate, LayerNormalization, MultiHeadAttention, 
  Reshape, Multiply, Softmax
)
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from utils import (
  encode_labels, check_gpu_availability, plot_loss_accuracytupian, 
  evaluate_model, plot_confusion_matrixtupian, plot_tsne, 
  plot_precision_recall_curve_multiclasstupian, plot_roc_curve_multiclasstupian, 
  AdjustLearningRateCallback, denoise2,count_labels,denoise2_iterative2,AdjustLearningRateCallback
)
from utils import plot_precision_recall_curve_multiclass,plot_roc_curve_multiclass2,calculate_g_mean,plot_confusion_matrix,plot_confusion_matrix2,plot_loss_accuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1DTranspose


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
check_gpu_availability()

GPU 可用


In [3]:
datafilename1 = "C:\\Users\\Administrator\\Desktop\\database\\cinc2017denoise.npz"
data1 = np.load(datafilename1, allow_pickle=True)
X_train, y_train, X_val, y_val, X_test, y_test = data1['ecgstrain'], data1['labelstrain'], data1['ecgsval'], data1['labelsval'], data1['ecgstest'], data1['labelstest']

In [4]:
y_train = encode_labels(y_train)
y_test = encode_labels(y_test)
y_val= encode_labels(y_val)
y_train = to_categorical(y_train, num_classes=4)
y_val=to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [5]:
from tensorflow.keras.layers import GlobalAveragePooling1D, Concatenate
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dropout, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size, strides):
    shortcut = x
    # 主卷积路径
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # 如果shortcut的形状与x不匹配，则调整shortcut
    if shortcut.shape[-1] != x.shape[-1] or shortcut.shape[-2] != x.shape[-2]:
        shortcut = Conv1D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 添加跳过连接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 31, strides=4, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # Residual blocks
    x = residual_block(x, 32, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x_row = GRU(256, return_sequences=True)(x)
    x_row= Dropout(0.1)(x_row)
    x_col = GRU(256, return_sequences=True)(tf.transpose(x, [0, 2, 1]))
    x_col = Dropout(0.1)(x_col)
    x_col_pooled = GlobalAveragePooling1D()(x_col)
    x_dense = Dense(128, activation='relu')(x_col_pooled)
    outputs = Dense(num_classes, activation='softmax')(x_dense)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
model = build_model((4500, 1), 4)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4500, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1125, 32)     1024        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1125, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 1125, 32)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
callback = AdjustLearningRateCallback(factor=0.1, patience=2, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=256, epochs=80, validation_data=(X_val, y_val), callbacks=[callback,early_stopping])

Epoch 1/80
55/55 [==============================] - 14s 70ms/step - loss: 0.4063 - accuracy: 0.5966 - val_loss: 0.4986 - val_accuracy: 0.3422
Epoch 2/80
55/55 [==============================] - 3s 54ms/step - loss: 0.3447 - accuracy: 0.6625 - val_loss: 0.6587 - val_accuracy: 0.3046
Epoch 3/80
55/55 [==============================] - 3s 52ms/step - loss: 0.3015 - accuracy: 0.7185 - val_loss: 0.8645 - val_accuracy: 0.1542
Reduced learning rate to 0.00010000000474974513.
Epoch 4/80
55/55 [==============================] - 3s 53ms/step - loss: 0.2669 - accuracy: 0.7605 - val_loss: 0.5784 - val_accuracy: 0.3580
Epoch 5/80
55/55 [==============================] - 3s 55ms/step - loss: 0.2592 - accuracy: 0.7721 - val_loss: 0.4050 - val_accuracy: 0.5597
Epoch 6/80
55/55 [==============================] - 3s 53ms/step - loss: 0.2525 - accuracy: 0.7788 - val_loss: 0.3158 - val_accuracy: 0.6986
Epoch 7/80
55/55 [==============================] - 3s 52ms/step - loss: 0.2474 - accuracy: 0.7863 - val

In [7]:
import numpy as np

def add_low_frequency_noise_multidim(data, snr, frequency_range=(0, 5), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [8]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,0)

In [9]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.5435898342756141
Recall: 0.436074100936239
F1 Score: 0.46018940230762895
Accuracy: 0.7333065164923572
Class 1 - Precision: 0.8, Recall: 0.29955947136563876, F1 Score: 0.43589743589743596
Class 2 - Precision: 0.7887077997671711, Recall: 0.9027315123251166, F1 Score: 0.8418763591177385
Class 3 - Precision: 0.5856515373352855, Recall: 0.5420054200542005, F1 Score: 0.5629838142153413
Class 4 - Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Class 1 Accuracy: 0.9292035398230089
Class 2 Accuracy: 0.7952534191472245
Class 3 Accuracy: 0.7502011263073209
Class 4 Accuracy: 0.9919549477071601


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,5)

In [11]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.549662672975173
Recall: 0.521922991372274
F1 Score: 0.5337300410840708
Accuracy: 0.7703137570394207
Class 1 - Precision: 0.7248677248677249, Recall: 0.6035242290748899, F1 Score: 0.6586538461538463
Class 2 - Precision: 0.8264615384615385, Recall: 0.8947368421052632, F1 Score: 0.8592450415866923
Class 3 - Precision: 0.6473214285714286, Recall: 0.5894308943089431, F1 Score: 0.6170212765957447
Class 4 - Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Class 1 Accuracy: 0.9428801287208367
Class 2 Accuracy: 0.8230088495575221
Class 3 Accuracy: 0.7827835880933226
Class 4 Accuracy: 0.9919549477071601


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
import numpy as np

def add_high_frequency_noise_multidim(data, snr, frequency_range=(0, 200), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [13]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,0)

In [14]:
evaluate_model(model,X_test_noisy,y_test)

Precision: 0.5249282970914176
Recall: 0.42476769821436255
F1 Score: 0.44557741641074977
Accuracy: 0.7308930008045053
Class 1 - Precision: 0.7283950617283951, Recall: 0.2599118942731278, F1 Score: 0.38311688311688313
Class 2 - Precision: 0.7872340425531915, Recall: 0.9120586275816123, F1 Score: 0.8450617283950617
Class 3 - Precision: 0.5840840840840841, Recall: 0.5271002710027101, F1 Score: 0.5541310541310541
Class 4 - Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Class 1 Accuracy: 0.9235720032180209
Class 2 Accuracy: 0.7980691874497184
Class 3 Accuracy: 0.7481898632341111
Class 4 Accuracy: 0.9919549477071601


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,5)

In [16]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.5516620432459972
Recall: 0.5179609506336693
F1 Score: 0.5317708100776353
Accuracy: 0.7739340305711987
Class 1 - Precision: 0.7228260869565217, Recall: 0.5859030837004405, F1 Score: 0.6472019464720195
Class 2 - Precision: 0.8238841978287093, Recall: 0.9100599600266489, F1 Score: 0.8648306426084204
Class 3 - Precision: 0.6599378881987578, Recall: 0.575880758807588, F1 Score: 0.6150506512301012
Class 4 - Precision: 0.0, Recall: 0.0, F1 Score: 0.0
Class 1 Accuracy: 0.9416733708769107
Class 2 Accuracy: 0.828238133547868
Class 3 Accuracy: 0.7860016090104586
Class 4 Accuracy: 0.9919549477071601


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
